## Experiments: Sensitive Analysis Hyperparameter RSAST:


It is runned RSAST in a set of UCR datasets with a predefined number of runs ("runs"). Then, it is selected a range ("range_total") between 1, 10, 100 and 10000 for the selected dataset.

In [1]:
from sast.sast import *
from sktime.datasets import load_UCR_UEA_dataset, tsc_dataset_names
from sktime.classification.kernel_based import RocketClassifier
import time
import pandas as pd
import os

/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/mass_ts/_mass_ts.py:17: UserWarning: GPU support will not work. You must pip install mass-ts[gpu].
  warnings.warn(


### Reading Datasets

In [25]:
# Set directory where the csv files are located
directory = '/home/nirojasvar/random_sast/ResultsByClassifier'

# Create an empty list to store the dataframes
dfs = []

# Loop through all files in the directory
for filename in os.listdir(directory):
    # Check if the file starts with "df_overall_tunning" and ends with ".csv"
    if filename.endswith(".csv"):
        # Read the csv file into a dataframe
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath)
        # Append the dataframe to the list
        df['filename']=filename
        dfs.append(df)

# Concatenate all the dataframes in the list into one dataframe
df_other_methods = pd.concat(dfs, ignore_index=True)
df_other_methods=df_other_methods[['filename','folds:','0']]
df_other_methods['method']=df_other_methods['filename'].str.split('_').str[0]
df_other_methods
df_other_methods=df_other_methods.rename(columns={'folds:':'dataset','0':'score'})
df_other_methods=df_other_methods[['dataset','score','method']]


In [26]:
df_other_methods['method'].unique()

array(['ROCKET', 'HIVE-COTEv1', 'BOSS', 'RISE', 'TSF', 'InceptionTime',
       'TS-CHIEF', 'S-BOSS', 'STC', 'ResNet', 'ProximityForest', 'WEASEL',
       'Catch22', 'cBOSS'], dtype=object)

In [27]:
# Set directory where the csv files are located
directory = '/home/nirojasvar/random_sast/ResultsRsast'

# Create an empty list to store the dataframes
dfs = []

# Loop through all files in the directory
for filename in os.listdir(directory):
    # Check if the file starts with "df_overall_tunning" and ends with ".csv"
    if filename.startswith("df_overall_tunning") and filename.endswith(".csv"):
        # Read the csv file into a dataframe
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath)
        # Append the dataframe to the list
        dfs.append(df)

# Concatenate all the dataframes in the list into one dataframe
df_result = pd.concat(dfs, ignore_index=True)

In [28]:
#df_result=pd.read_csv("df_overall_tunning_SmoothSubspace3.csv")
df_result['dataset_name'].unique()


array(['SmoothSubspace', 'GunPoint', 'ItalyPowerDemand', 'ShapeletSim',
       'Chinatown', 'BirdChicken', 'SyntheticControl'], dtype=object)

In [29]:
df_result.columns.unique()

Index(['Unnamed: 0', 'accuracy', 'time', 'cweights_time', 'fsubsequence_time',
       'tdataset_time', 'tclassifier_time', 'dataset_name', 'classifier_name',
       'rpoint', 'nb_per_class', 'method', 'len_method'],
      dtype='object')

### Hyperparameter Tunning RSAST 

#### Accuracy: Subsequence Lenght Method 

In [30]:
# create a pivot table with the mean of score by dataset
len_method_vs_ds=pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['dataset_name'], aggfunc='mean')
len_method_vs_ds=np.transpose(len_method_vs_ds)#[['ACF','PACF']]
len_method_vs_ds=len_method_vs_ds.reset_index()
df_rocket=df_other_methods[df_other_methods["method"]=="ROCKET"]
merged_df = len_method_vs_ds.merge(df_rocket,left_on='dataset_name', right_on='dataset',  how='left')
merged_df=merged_df.drop('method',axis=1)
merged_df

,dataset_name,ACF,ACF&PACF,Max ACF,Max PACF,None,PACF,dataset,score
0,BirdChicken,0.776667,0.830000,0.713333,0.755333,0.708667,0.837333,BirdChicken,0.900000
1,Chinatown,0.950865,0.962177,0.938503,0.960233,0.947833,0.963460,Chinatown,0.982507
2,GunPoint,0.778933,0.971111,0.784889,0.947556,0.918311,0.964978,GunPoint,1.000000
3,ItalyPowerDemand,0.890897,0.903026,0.879585,0.874921,0.885079,0.897933,ItalyPowerDemand,0.968902
4,ShapeletSim,0.944296,0.891037,0.895704,0.734593,0.631259,0.881037,ShapeletSim,1.000000
5,SmoothSubspace,0.816000,0.863022,0.821333,0.849511,0.827556,0.861689,SmoothSubspace,0.980000
6,SyntheticControl,0.358533,0.318400,0.311067,0.292400,0.292800,0.352933,SyntheticControl,1.000000


In [9]:
# create a pivot table with the mean of score by number of random points
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['rpoint'], aggfunc='mean')

rpoint,1,10,30,50,100
len_method,,,,,
ACF,0.765990,0.852075,0.837731,0.858026,0.852364
ACF&PACF,0.836206,0.879602,0.875076,0.886649,0.885497
Max ACF,0.733750,0.822508,0.814821,0.845642,0.838835
Max PACF,0.754438,0.838867,0.830834,0.844568,0.852040
None,0.742136,0.807166,0.792093,0.812077,0.806769
PACF,0.832301,0.872130,0.884096,0.884414,0.888171


In [10]:
# create a pivot table with the mean of score by number of instances per class
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10,30,50,100
len_method,,,,,
ACF,0.748682,0.833416,0.842445,0.873568,0.868076
ACF&PACF,0.832018,0.862242,0.889085,0.891314,0.888372
Max ACF,0.700361,0.827768,0.818850,0.850920,0.857656
Max PACF,0.733720,0.836699,0.829711,0.862148,0.858470
None,0.690944,0.806065,0.807236,0.824658,0.831337
PACF,0.830311,0.865915,0.888457,0.889937,0.886491


#### Time: Subsequence Lenght Method 

In [11]:
# create a pivot table with the mean of the time by dataset
pd.pivot_table(df_result, values='time', index=['len_method'],columns=['dataset_name'], aggfunc='mean')

dataset_name,BirdChicken,Chinatown,GunPoint,ItalyPowerDemand,ShapeletSim,SmoothSubspace,SyntheticControl
len_method,,,,,,,
ACF,81.630738,0.143859,39.144550,0.826132,7.645123,2.081739,128.364227
ACF&PACF,297.280662,0.444094,88.200498,2.032732,222.211859,3.407822,410.825064
Max ACF,2.745482,0.145839,4.153232,0.839920,2.640422,2.087500,45.500390
Max PACF,107.885490,0.162721,3.936128,0.915300,124.217464,2.404325,30.050111
None,1.853694,0.133453,2.529203,0.829915,1.836391,2.064745,27.938759
PACF,208.928276,0.387296,48.907430,2.043757,215.296044,3.382209,261.462549


In [12]:
# create a pivot table with the mean of the time by number of instances per class
pd.pivot_table(df_result, values='time', index=['len_method'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10,30,50,100
len_method,,,,,
ACF,2.277378,23.317501,34.101064,38.968926,38.908987
ACF&PACF,11.756699,114.625243,143.246852,161.764617,161.121869
Max ACF,0.320612,2.457961,5.059440,7.068554,7.024375
Max PACF,4.518146,44.861220,53.702551,47.519943,46.401928
None,0.243456,1.692588,3.424845,4.581256,4.710740
PACF,9.060306,91.436185,109.427573,120.126856,116.869498


In [13]:
# create a pivot table with the mean of the time by number of random points
pd.pivot_table(df_result, values='time', index=['len_method'],columns=['rpoint'], aggfunc='mean')

rpoint,1,10,30,50,100
len_method,,,,,
ACF,0.832222,7.356535,21.608321,35.893236,71.883541
ACF&PACF,37.041986,55.911648,101.214066,149.019189,249.328391
Max ACF,0.214663,1.238401,3.434945,5.756041,11.286892
Max PACF,34.008144,33.716794,40.698942,42.778541,45.801368
None,0.145160,0.833648,2.286639,3.821309,7.566128
PACF,35.982256,47.016232,82.549200,109.397195,171.975535


#### Variance: Subsequence Lenght Method 

In [14]:
# create a pivot table with the variance of the accuracy by number of instances per class
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['nb_per_class'], aggfunc='var')

nb_per_class,1,10,30,50,100
len_method,,,,,
ACF,0.036603,0.032316,0.032050,0.017476,0.022902
ACF&PACF,0.028527,0.029359,0.020281,0.020553,0.024136
Max ACF,0.039083,0.029548,0.041261,0.029703,0.019831
Max PACF,0.044391,0.035011,0.040747,0.025730,0.026250
None,0.044320,0.039444,0.040755,0.035730,0.036114
PACF,0.025576,0.027609,0.018536,0.021128,0.023237


In [16]:
# create a pivot table with the variance of the accuracy by number of random points
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['rpoint'], aggfunc='var')

rpoint,1,10,30,50,100
len_method,,,,,
ACF,0.038729,0.025486,0.031314,0.021284,0.028776
ACF&PACF,0.033733,0.022189,0.026390,0.020832,0.020602
Max ACF,0.043946,0.033985,0.039896,0.023994,0.025957
Max PACF,0.045457,0.032659,0.038667,0.032149,0.027920
None,0.050112,0.038470,0.045203,0.035518,0.037084
PACF,0.030158,0.026482,0.020542,0.021377,0.017972


#### Another Results

In [17]:
df_result[['dataset_name','len_method','classifier_name']].describe()

,dataset_name,len_method,classifier_name
count,2850,2850,2850
unique,7,6,150
top,SmoothSubspace,ACF&PACF,ACF&PACF: n_random_points=1 nb_inst_per_class=1
freq,450,475,19


In [18]:
pd.pivot_table(df_result, values='time', index=['dataset_name'], aggfunc='sum')/(60*60)

,time
dataset_name,
BirdChicken,14.590090
Chinatown,0.029526
GunPoint,3.893147
ItalyPowerDemand,0.155995
ShapeletSim,11.955152
SmoothSubspace,0.321424
SyntheticControl,6.278758


In [19]:
df_result['time'].sum()/(60*60)

37.22409189270602

#### Generate CD Diagram Tunning Hyperparameter

In [22]:
#create dataset to generate cd diagram
df_perf=pd.pivot_table(df_result, values='accuracy', index=['classifier_name','dataset_name'], aggfunc='mean')
df_perf=df_perf.reset_index()
#df_perf.classifier_name.unique()

In [ ]:
from cd_diagram.cd_function import *
#generate hyperparameter tuning cd diagrams
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^ACF&PACF:')], labels=True, title="ACF&PACF comparison", name_file='cd-diagram_ACF&PACF.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^PACF:')], labels=True, title="PACF comparison", name_file='cd-diagram_PACF.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^ACF:')], labels=True, title="ACF comparison", name_file='cd-diagram_ACF.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^None:')], labels=True, title="None comparison", name_file='cd-diagram_None.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^Max ACF:')], labels=True, title="Max ACF comparison", name_file='cd-diagram_MaxACF.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^Max PACF:')], labels=True, title="Max PACF comparison", name_file='cd-diagram_MaxPACF.png')
_, average_ranks, _ = wilcoxon_holm(df_perf=df_perf)
min_rank= min(average_ranks)
average_ranks=pd.DataFrame(average_ranks)


In [32]:
average_ranks

,0
None: n_random_points=30 nb_inst_per_class=1,136.285714
None: n_random_points=10 nb_inst_per_class=1,132.357143
Max ACF: n_random_points=30 nb_inst_per_class=1,132.000000
Max ACF: n_random_points=10 nb_inst_per_class=1,130.928571
Max ACF: n_random_points=1 nb_inst_per_class=1,129.714286
...,...
ACF&PACF: n_random_points=50 nb_inst_per_class=100,41.214286
Max PACF: n_random_points=30 nb_inst_per_class=30,40.642857
ACF&PACF: n_random_points=30 nb_inst_per_class=50,40.428571
ACF&PACF: n_random_points=100 nb_inst_per_class=30,37.642857


In [31]:
average_ranks[average_ranks[0]==min_rank][0].index

Index(['ACF&PACF: n_random_points=100 nb_inst_per_class=50'], dtype='object')

In [ ]:
#generate example cd diagrams
draw_cd_diagram(pd.read_csv('example_cd_diagram.csv'), labels=True, title="Example CD Diagram", name_file='cd-diagram_example.png')

#### Focus on most accurate lenght method

##### Overall Accuracy

In [33]:
# filter by method with ACF&PACF
# create a pivot table with the mean of score by hyperparameter
df_result_acc=df_result[df_result["len_method"]=="ACF&PACF"]
pd.pivot_table(df_result_acc, values='accuracy', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10,30,50,100
rpoint,,,,,
1,0.757272,0.797292,0.878385,0.879132,0.868949
10,0.847998,0.878287,0.891467,0.887579,0.892680
30,0.819710,0.879742,0.885958,0.896072,0.893901
50,0.867370,0.878741,0.894084,0.897761,0.895290
100,0.867740,0.877146,0.895530,0.896027,0.891043


In [34]:
# create a pivot table with the variatioon of score by hyperparameter
pd.pivot_table(df_result_acc, values='accuracy', index=['rpoint'],columns=['nb_per_class'], aggfunc='var')

nb_per_class,1,10,30,50,100
rpoint,,,,,
1,0.052831,0.048220,0.017478,0.014542,0.029957
10,0.023127,0.021033,0.020629,0.025095,0.024541
30,0.039906,0.031325,0.020808,0.020848,0.020704
50,0.011500,0.020839,0.025769,0.025486,0.024470
100,0.012616,0.026332,0.021022,0.021098,0.025860


##### Overall time complexity

In [40]:
# create a pivot table with the mean of time spent in minutes
pd.pivot_table(df_result_acc, values=['time','cweights_time','fsubsequence_time','tdataset_time','tclassifier_time'], index=['dataset_name'], aggfunc='mean')/60

,cweights_time,fsubsequence_time,tclassifier_time,tdataset_time,time
dataset_name,,,,,
BirdChicken,0.001775,1.845793,0.000515,3.106501,4.954678
Chinatown,0.000160,0.001823,0.000126,0.005264,0.007402
GunPoint,0.000717,0.042117,0.001210,1.425890,1.470008
ItalyPowerDemand,0.000144,0.003278,0.000300,0.030122,0.033879
ShapeletSim,0.002136,2.069743,0.000366,1.631211,3.703531
SmoothSubspace,0.000118,0.003916,0.000706,0.052012,0.056797
SyntheticControl,0.000782,0.051287,0.449126,6.345621,6.847084


In [43]:
# create a pivot table with the mean of time spent in seconds by random points and instances per class
pd.pivot_table(df_result_acc, values='time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10,30,50,100
rpoint,,,,,
1,5.064929,43.357442,48.872824,45.987409,41.927325
10,6.545510,60.061166,71.888804,72.759014,68.303747
30,9.932378,96.197427,123.403307,137.651269,138.885948
50,14.537053,141.176949,178.327223,191.741511,219.313208
100,22.703626,232.333229,293.742100,360.683883,337.179119


##### Calculate weights time complexity

In [11]:
pd.pivot_table(df_result_acc, values='cweights_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10,30,50,100
rpoint,,,,,
1,0.028192,0.028556,0.027600,0.028790,0.030082
10,0.029349,0.025693,0.025898,0.029354,0.024608
30,0.022716,0.031543,0.029285,0.034425,0.029611
50,0.029784,0.026948,0.028282,0.030720,0.031881
100,0.029638,0.027652,0.028193,0.030118,0.025571


##### Finding subsequences time complexity

In [12]:
pd.pivot_table(df_result_acc, values='fsubsequence_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10,30,50,100
rpoint,,,,,
1,0.570715,0.450227,0.526023,0.500756,0.546674
10,0.467190,0.462596,0.496268,0.511135,0.533021
30,0.549582,0.544082,0.676318,0.758593,0.745421
50,0.694216,0.721292,0.873402,0.976438,1.038515
100,0.948055,1.015550,1.283563,1.370078,1.323146


##### Transform Dataset time complexity

In [13]:
pd.pivot_table(df_result_acc, values='tdataset_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10,30,50,100
rpoint,,,,,
1,0.428162,0.123954,0.143920,0.156997,0.153823
10,1.133522,1.208636,1.386036,1.510926,1.508498
30,3.357759,3.566634,4.144730,4.521114,4.506077
50,5.545827,5.986823,6.918355,7.589233,7.518632
100,11.129992,11.864132,13.768211,15.064077,14.992503


##### Classifier time complexity

In [42]:
pd.pivot_table(df_result_acc, values='tclassifier_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10,30,50,100
rpoint,,,,,
1,0.021719,0.024943,0.019637,0.024128,0.022337
10,0.012794,0.030270,0.034013,0.035426,0.038185
30,0.025469,0.038695,0.073311,0.168051,1.924190
50,0.028331,0.047768,0.192097,0.136022,31.030929
100,0.019695,0.085454,0.996914,0.982187,0.208157
